In [2]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
from datetime import datetime
from dateutil import tz
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker


import pynwb
from pynwb import register_class
from pynwb.form.utils import docval, getargs, popargs, call_docval_func

import nspike_helpers as ns 
import query_helpers as qu

mdates.rcParams.update({'date.autoformatter.microsecond': '%H:%M:%S.%f'})

/Users/ericmiller/anaconda/envs/pynwb/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Overview
* **FL_BehavioralEpochs** - A DynamicTable where each row contains data from an epoch, possibly including:

    * **name** - Name of the epoch
    * **position** - SpatialSeries for animal position during the epoch, possibly including:
        * {(x1, y1)...} - 2D position vs time, linked to a 2D track geometry (e.g. in the FL_BehavioralWorld object, see below)
        * {d1, ...} - 1D distance vs time, linked to a 1D linearized track geometry   
    * **head_dir** - SpatialSeries for animal head direction during the epoch
    * **speed** - TimeSeries for animal speed during the epoch
    * **FL_BehavioralTask** - Container for data describing the task/apparatus for the epoch, possibly including:
        * **FL_BehavioralFSM** - Finite State Machine representing discrete, non-overlapping states of the behavior (e.g. on segment 3, not at well)
        * **FL_BehavioralEvents** - DynamicTable of possibly-overlapping behavioral events (e.g. cue light on, reward delivery on)
        * **FL_BehavioralWorld** - collection of geometries describing the behavioral world, possibly including:
            * 1D linearized representation of the track
            * 2D geometries representing track borders
            * 2D or 3D geometries representing the experimental room
            * geoemtries for objects, cues, or other elements of the world (e.g. locations of wells, cues, levers)
    

### Spec for a Behavioral Task group
An object for representing a behavioral task, experimental apparatus,

Contains: 
* **FL_BehavioralWorld** - geometry describing the spatial and operant world of this task (e.g. polygons, lines, points representing 1D and 2D track geometry, 3D room, notable objects or cues). These world geometries can then be used as reference frames when describing animal position.
* **FL_BehavioralFSM** - discrete, non-overlapping states of animal behavior (e.g. open-field no well, open-field at well, arm 4 no well, arm 4 at well, ...)
* **FL_BehavioralEvents** - DynamicTable of possibly-overlapping instances of experimental/behavioral events (e.g. reward delivered, cue light on)



In [3]:
world_spec = pynwb.spec.NWBGroupSpec(name='world',
                                               doc='Collection of geometries (1D, 2D, or 3D) describing the behavioral environment',
                                               neurodata_type_inc='NWBContainer',
                                               neurodata_type_def='FL_BehavioralWorld')

fsm_spec = pynwb.spec.NWBGroupSpec(name='fsm',
                                               doc='Finite state machine describing non-overlapping behavioral states',
                                               neurodata_type_inc='NWBContainer',
                                              neurodata_type_def='FL_BehavioralFSM')

events_spec = pynwb.spec.NWBGroupSpec(name='events',
                                               doc='DynamicTable of describing possibly-overlapping behavioral/experimental events',
                                               neurodata_type_inc='DynamicTable',
                                                neurodata_type_def='FL_BehavioralEvents')

behavioral_task_spec = pynwb.spec.NWBGroupSpec(name='FL_BehavioralTask',
                                               doc='Container of objects for describing a behavioral task',
                                               groups=[world_spec, fsm_spec, events_spec],
                                               neurodata_type_inc='NWBContainer',
                                               neurodata_type_def='FL_BehavioralTask')

### Spec for an Epochs DynamicTable
A table of data for each behavioral epoch.

In [4]:
epoch_datasets = [pynwb.spec.NWBDatasetSpec(name='epoch_name',
                                            doc='Name for this epoch',
                                            dtype='text',
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                  pynwb.spec.NWBDatasetSpec(name='task',
                                            doc='Task used in this epoch',
                                            dtype=pynwb.spec.NWBRefSpec('FL_BehavioralTask', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                  pynwb.spec.NWBDatasetSpec(name='position', 
                                            doc='Animal spatial position during an epoch', 
                                            dtype=pynwb.spec.NWBRefSpec('SpatialSeries', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                  pynwb.spec.NWBDatasetSpec(name='speed', 
                                            doc='Animal speed during an epoch', 
                                            dtype=pynwb.spec.NWBRefSpec('TimeSeries', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData'),
                  pynwb.spec.NWBDatasetSpec(name='head_dir', 
                                            doc='Animal head direction during an epoch',
                                            dtype=pynwb.spec.NWBRefSpec('SpatialSeries', 'object'),
                                            quantity='?',
                                            neurodata_type_inc='VectorData')]

behavioral_epochs_spec = pynwb.spec.NWBGroupSpec(name='FL_BehavioralEpochs',
                                                 doc='DynamicTable for describing behavioral epochs',
                                                 neurodata_type_inc='DynamicTable', 
                                                 neurodata_type_def='FL_BehavioralEpochs',
                                                 datasets=epoch_datasets,
                                                 quantity="?")

### Build a namespace and export to YAML

In [5]:
ns_path = "franklab.namespace.yaml"
ext_path = "franklab.extensions.yaml"

ns_builder = pynwb.spec.NWBNamespaceBuilder('Extension for use in the Frank Lab', "franklab")

ns_builder.add_spec(ext_path, behavioral_task_spec)
ns_builder.add_spec(ext_path, behavioral_epochs_spec)

ns_builder.export(ns_path)

### Load our namespace

In [6]:
pynwb.load_namespaces(ns_path)

('franklab',)

### Auto-generate a Python class for our extensions

#### Currently does not work...
Auto-generation seems to not work with DynamicTables yet.
It correctly creates a class inheriting from DynamicTable, but it does not implement the columns
in the "columns" class variable or the docstring in "add_row()". Recall that in the Units DynamicTable,
a new method "add_unit()" was created with a docstring for the correct columns (e.g. spiketimes, obs_intervals).
A similar "add_epoch()" method is not auto-generated.

In [7]:
from pynwb import get_class
# FL_BehavioralEpochs = get_class('FL_BehavioralEpochs', 'franklab')  # This is currently not working
FL_BehavioralWorld = get_class('FL_BehavioralWorld', 'franklab')
FL_BehavioralFSM = get_class('FL_BehavioralFSM', 'franklab')
FL_BehavioralEvents = get_class('FL_BehavioralEvents', 'franklab')
# FL_BehavioralTask = get_class('FL_BehavioralTask', 'franklab')      # This is currently not working

### Manually create a class for FL_BehavioralEpochs

In [8]:
@register_class('FL_BehavioralEpochs', 'franklab')
class FL_BehavioralEpochs(pynwb.core.DynamicTable):
    """
    Data for behavioral epochs.
    """

    # Setting index to True throws an error if I try to store non-iterable types, such as SpatialSeries
    __columns__ = (
        {'name': 'epoch_name', 'description': 'Name for this epoch', 'index': False},
        {'name': 'position', 'description': 'Animal spatial position during an epoch', 'index': False},
        {'name': 'speed', 'description': 'Animal speed during an epoch', 'index': False},
        {'name': 'head_dir', 'description': 'Animal head direction during an epoch', 'index': False},
        {'name': 'task', 'description': 'Behavioral task for this epoch', 'index': False}
    )

    @docval({'name': 'name', 'type': str, 'doc': 'Name of this FL_BehavioralEpochs table', 'default': 'FL_BehavioralEpochs'},
            {'name': 'id', 'type': ('array_data', pynwb.core.ElementIdentifiers),
             'doc': 'the identifiers for the epochs stored in this interface', 'default': None},
            {'name': 'columns', 'type': (tuple, list), 'doc': 'the columns in this table', 'default': None},
            {'name': 'colnames', 'type': 'array_data', 'doc': 'the names of the columns in this table',
             'default': None},
            {'name': 'description', 'type': str, 'doc': 'a description of what is in this FL_BehavioralEpochs table', 'default': None})
    def __init__(self, **kwargs):
        if kwargs.get('description', None) is None:
            kwargs['description'] = ""
        call_docval_func(super(FL_BehavioralEpochs, self).__init__, kwargs)

    @docval({'name': 'epoch_name', 'type': str, 'doc': 'Name for this epoch', 'default': None},
            {'name': 'position', 'type': pynwb.behavior.SpatialSeries, 'doc': 'Animal spatial position during an epoch', 'default': None},
            {'name': 'speed', 'type': pynwb.base.TimeSeries, 'doc': 'Animal speed during an epoch', 'default': None},
            {'name': 'head_dir', 'type': pynwb.behavior.SpatialSeries, 'doc': 'Animal head direction during an epoch', 'default': None},
            {'name': 'task', 'type': 'FL_BehavioralTask', 'doc': 'Behavioral task for this epoch', 'default': None},
            {'name': 'id', 'type': int, 'default': None, 'help': 'the id for each unit'},
            allow_extra=True)
    def add_epoch(self, **kwargs):
        """
        Add an epoch to this table
        """
        super(FL_BehavioralEpochs, self).add_row(**kwargs)


### Manually create a class for FL_BehavioralTask

In [9]:
@register_class('FL_BehavioralTask', 'franklab')
class FL_BehavioralTask(pynwb.core.NWBContainer):
    __nwbfields__ = ('name', 'world', 'fsm', 'events')

    @docval({'name': 'name', 'type': str, 'doc': 'who names a potato?', 'default': 'FL_BehavioralTask'},
            {'name': 'world', 'type': 'FL_BehavioralWorld', 'doc': 'Collection of geometries (1D, 2D, or 3D) describing the behavioral environment', 'default': None},
            {'name': 'fsm', 'type': 'FL_BehavioralFSM', 'doc': 'Finite state machine describing non-overlapping behavioral states', 'default': None},
            {'name': 'events', 'type': 'FL_BehavioralEvents', 'doc': 'DynamicTable of describing possibly-overlapping behavioral/experimental events', 'default': None})
    def __init__(self, **kwargs):
        super(FL_BehavioralTask, self).__init__(name=kwargs['name'])
        self.world = kwargs['world']
        self.fsm = kwargs['fsm']
        self.events = kwargs['events']

### Extract behavioral data

In [10]:
# Session-specific params
data_dir = os.path.expanduser('~/Data/FrankData/kkay/Bon')
anim = 'Bon' 
prefix = anim.lower()
day = 3 # below we'll code date as 2006-Jan-'Day'

# Calculate the POSIX timestamp when Nspike clock = 0 (seconds)
dataset_zero_time = datetime(2006, 1, day, 12, 0, 0, tzinfo=tz.gettz('US/Pacific'))
NSpike_posixtime_offset = dataset_zero_time.timestamp()

# NOTE that day_inds is 0 based
time_list = {}
nwb_epoch = {}
pos_files = ns.get_files_by_day(data_dir, prefix, 'pos')
task_files = ns.get_files_by_day(data_dir, prefix, 'task')

mat = ns.loadmat_ff(task_files[day], 'task')
task_struct = mat[day]

mat = ns.loadmat_ff(pos_files[day], 'pos')
pos_struct = mat[day]

# create position, direction and speed
position_list = []
direction_list = []
speed_list = []
time_list = []

# Assume field order: (time,x,y,dir,vel)
(time_idx, x_idx, y_idx, dir_idx, vel_idx) = range(5)

for epoch_num, pos_epoch in pos_struct.items():

    # convert times to POSIX time
    timestamps = pos_epoch['data'][:,time_idx] + NSpike_posixtime_offset

    # TODO: create a shared TimeSeries for timestamps, across all behavioral timeseries
    # ?? timestamps_obj = pynwb.TimeSeries(timestamps=timestamps...)

    # collect times of epoch start and end
    time_list.append([timestamps[0], timestamps[-1]])

    m_per_pixel = pos_epoch['cmperpixel'][0,0]/100 # NWB wants meters per pixel

    # we can also create new SpatialSeries for the position, direction and velocity information
    #NOTE: Each new spatial series has to have a unique name.
    position_list.append(pynwb.behavior.SpatialSeries(name='Position d%d e%d' % (day, epoch_num), 
                              timestamps = timestamps,
                              data=pos_epoch['data'][:, (x_idx, y_idx)] * m_per_pixel,
                              reference_frame='corner of video frame',
                              #conversion=m_per_pixel,
                              #unit='m'
                              )) # *after* conversion

    direction_list.append(pynwb.behavior.SpatialSeries(name='Head Direction d%d e%d'% (day, epoch_num), 
                              timestamps=timestamps,
                              data=pos_epoch['data'][:, dir_idx],
                              reference_frame='0=facing top of video frame (?), positive clockwise (?)',
                              #unit='radians'
                              ))

    speed_list.append(pynwb.base.TimeSeries(name='Speed d%d e%d' % (day, epoch_num),
                             timestamps=timestamps,
                             data=pos_epoch['data'][:, vel_idx] * m_per_pixel,
                             unit='m/s',
                             #conversion=m_per_pixel,
                             description='smoothed movement speed estimate'))
time_list = np.asarray(time_list)

### Implement an Epochs table using all of our extensions

In [11]:
# Define a World for each task
w_track_1 = FL_BehavioralWorld(name='W-track, Room 1')
sleep_box_1 = FL_BehavioralWorld(name='Sleep box, Room 1')

# Define an FSM for each task
w_track_fsm = FL_BehavioralFSM(name='W-track FSM')
sleep_box_fsm = FL_BehavioralFSM(name='Sleep box FSM')
                                 
# Events tables for each epoch                      
num_epochs = len(pos_struct)
events_list = [FL_BehavioralEvents(name='Events epoch ' + str(i), 
                                   description='Events for epoch ' + str(i)) for i in range(num_epochs)]

# Epochs table
epochs = FL_BehavioralEpochs(name='Epochs for Bon day 3')
for epoch_idx in range(num_epochs):       
    # Task object for this epoch
    task = FL_BehavioralTask(name='Task for epoch ' + str(epoch_idx),
                             world=w_track_1, 
                             fsm=w_track_fsm, 
                             events=events_list[epoch_idx])
    # Add a row to the table for this epoch
    epochs.add_epoch(epoch_name='Epoch ' + str(epoch_idx), 
                     position=position_list[epoch_idx], 
                     speed=speed_list[epoch_idx], 
                     head_dir=direction_list[epoch_idx],
                     task=task)
    

### Inspect our Epochs table

In [26]:
print("An Epochs DynamicTable\n--------------")
print(epochs)
print('\n')

print("Position in epoch 2\n------------")
print(epochs[1, 'position'])
print('\n')

print("Speed in epoch 4\n------------")
print(epochs[3, 'speed'])
print('\n')

print("Head direction in epoch 1\n------------")
print(epochs[0, 'head_dir'])
print('\n')

print("Task for the final epoch\n----------")
print(">>> Currently the sub-groups don't contain any real data.")
print(">>> For some reason inspecting task shows the sub-groups as being None, when in fact they are not...")
print(epochs[-1, 'task'])
print(">>> But we can access them individually...")
print(epochs[-1, 'task'].world)
print(epochs[-1, 'task'].fsm)
print(epochs[-1, 'task'].events)
print('\n')

An Epochs DynamicTable
--------------

FL_BehavioralEpochs <class '__main__.FL_BehavioralEpochs'>
Fields:
  colnames: ('epoch_name', 'position', 'speed', 'head_dir', 'task')
  columns: (
epoch_name <class 'pynwb.core.VectorData'>
Fields:
  description: Name for this epoch
, 
position <class 'pynwb.core.VectorData'>
Fields:
  description: Animal spatial position during an epoch
, 
speed <class 'pynwb.core.VectorData'>
Fields:
  description: Animal speed during an epoch
, 
head_dir <class 'pynwb.core.VectorData'>
Fields:
  description: Animal head direction during an epoch
, 
task <class 'pynwb.core.VectorData'>
Fields:
  description: Behavioral task for this epoch
)
  description: 
  id: None



Position in epoch 2
------------

Position d3 e2 <class 'pynwb.behavior.SpatialSeries'>
Fields:
  comments: no comments
  conversion: 1.0
  data: [[1.93995 1.1583 ]
 [1.9359  1.1502 ]
 [1.93185 1.14615]
 ...
 [1.62    0.74115]
 [1.6119  0.73305]
 [1.60785 0.73305]]
  description: no description
